In [ ]:
import aiohttp
import asyncio
from datetime import datetime
import requests
import base64
import asyncio
from pathlib import Path
import time
import os

scenes_resp = VideoGen(
    story_them="A boy in a jungle with his cat and dog",
    scene_list=[
        ScenesList(
            id="2",
            scene="Jungle Swing",
            description="Leo swings on a thick jungle vine, with Whiskers perched on his head and Buddy running below.",
            narration="Once upon a time, in a lush green jungle, lived a boy named Leo. He loved to swing on vines with his best friends, Whiskers the cat and Buddy the dog. Whee! Up and down they went!",
            img_prompt="Create a detailed, photorealistic image of the following scene:\n            Leo swings on a thick jungle vine, with Whiskers perched on his head and Buddy running below.\n\n            **Main Characters**:\n            Leo - A young boy, around 7 years old, with bright, curious eyes, messy brown hair, and wearing simple khaki shorts and a green t-shirt. He is barefoot., Adventurous, kind, and a bit clumsy, with a strong bond with his pets.\n            **Supporting Characters**:\n            Whiskers - A fluffy ginger cat with green eyes and a playful demeanor. - Mischievous, agile, and fiercely loyal to Leo., Buddy - A golden retriever with floppy ears and a wagging tail. - Friendly, protective, and loves to play fetch.\n            **Objects**:\n            A thick, sturdy jungle vine hanging from a tall tree; dense green foliage; sunlight filtering through the leaves.\n            **Mood & Lighting**: Cinematic, immersive atmosphere with realistic lighting to match the scene's emotions.\n            The illustration should capture the story’s essence and atmosphere.",
            supporting_characters=[],
            main_characters=[],
            object_description=None,
            audio_path=None,
            image_path=None,
            video_path=None,
            combine_audiovideo_path=None,
            audio_duration=None,
        ),
        ScenesList(
            id="5",
            scene="The Lesson of Kindness",
            description="Mama Elephant takes Ravi back to his village, teaching him the importance of kindness.",
            narration="Mama Elephant gave Ravi a gentle ride back to his village. She reminded him that even the smallest act of kindness can make a big difference in the world. Ravi smiled, knowing he would always remember to be kind to all creatures, big and small.",
            img_prompt="Create a detailed, photorealistic image of the following scene:\n        Mama Elephant takes Ravi back to his village, teaching him the importance of kindness.\n\n        **Main Characters**:\n        Ravi - A small boy with bright, curious eyes, dark hair, and tanned skin. He wears a simple, slightly tattered brown loincloth., Adventurous, kind-hearted, resourceful, and brave. He is curious about the jungle and respectful of its creatures.\n        **Supporting Characters**:\n        Mama Elephant - A large, gentle elephant with wrinkled gray skin and long, graceful tusks. - Protective, wise, and nurturing. She cares deeply for her calf and the well-being of the jungle.\n        **Objects**:\n        A small village with thatched huts, Mama Elephant standing near the village, Ravi smiling, a setting sun.\n        **Mood & Lighting**: Cinematic, immersive atmosphere with realistic lighting to match the scene's emotions.\n        The illustration should capture the story’s essence and atmosphere.",
            supporting_characters=[],
            main_characters=[],
            object_description=None,
            audio_path=None,
            image_path=None,
            video_path=None,
            combine_audiovideo_path=None,
            audio_duration=None,
        ),
    ],
)

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp-image-generation:generateContent?key={os.getenv('GEMINI_API_KEY')}"


def current_time_with_millis():
    now = datetime.now()
    return now.strftime("%H:%M:%S") + f":{int(now.microsecond / 1000):03}"


async def generate_images(session, sce, outputfolder, ctx):
    print(f"current time {time.ctime()} for generate image id : {sce.id}")
    sce_id = sce.id
    index = int(sce_id) - 1
    prompt = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"You are an expert in generating **realistic** vertical images (1080x1920)...{sce.img_prompt}"
                    }
                ]
            }
        ],
        "generationConfig": {"responseModalities": ["Text", "Image"]},
    }
    payload = json.dumps(prompt)
    headers = {"Content-Type": "application/json"}

    async with session.post(url, data=payload, headers=headers) as resp:
        if resp.status == 200:
            data = await resp.json()
            base64_image = data["candidates"][0]["content"]["parts"][1]["inlineData"][
                "data"
            ]
            image_bytes = base64.b64decode(base64_image)
            final_path = f"{outputfolder}/image_{sce_id}.png"
            os.makedirs(os.path.dirname(final_path), exist_ok=True)
            with open(final_path, "wb") as f:
                f.write(image_bytes)

            print(f"successfully Time: {current_time_with_millis()}")
        else:
            print(
                f"Error while generating images: Error {resp.status} - Time: {current_time_with_millis()}"
            )


outputfolder = "armanfolder"
os.makedirs(outputfolder, exist_ok=True)


async def main():
    ctx = scenes_resp
    async with aiohttp.ClientSession() as session:
        tasks = [
            generate_images(session, scene, outputfolder, ctx)
            for scene in scenes_resp.scene_list
        ]
        await asyncio.gather(*tasks)


asyncio.run(main())